In [1]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]="a"
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)
  


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [6]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [7]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)

(91795, 8)


In [8]:
emb_dim=64
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.Conv1D(filters=256, kernel_size=6, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Conv1D(filters=128, kernel_size=6, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
])
emb_model.compile(optimizer='adam', loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 64)           16000064  
_________________________________________________________________
conv1d (Conv1D)              (None, 195, 256)          98560     
_________________________________________________________________
dropout (Dropout)            (None, 195, 256)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 195, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 190, 128)          196736    
_________________________________________________________________
dropout_1 (Dropout)          (None, 190, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 190, 128)          0

In [9]:
emb_model.fit(padded_docs, dummy_y,batch_size=256,epochs=15, verbose=1,validation_split=0.1)

Epoch 1/15
323/323 [==============================] - 33s 103ms/step - loss: 0.7334 - acc: 0.7371 - val_loss: 0.2872 - val_acc: 0.9184
Epoch 2/15
323/323 [==============================] - 33s 101ms/step - loss: 0.1542 - acc: 0.9560 - val_loss: 0.2834 - val_acc: 0.9234
Epoch 3/15
323/323 [==============================] - 33s 101ms/step - loss: 0.0512 - acc: 0.9854 - val_loss: 0.3740 - val_acc: 0.9190
Epoch 4/15
323/323 [==============================] - 33s 101ms/step - loss: 0.0273 - acc: 0.9920 - val_loss: 0.4879 - val_acc: 0.9208
Epoch 5/15
323/323 [==============================] - 33s 101ms/step - loss: 0.0200 - acc: 0.9941 - val_loss: 0.5035 - val_acc: 0.9155
Epoch 6/15
323/323 [==============================] - 33s 101ms/step - loss: 0.0203 - acc: 0.9941 - val_loss: 0.5175 - val_acc: 0.9234
Epoch 7/15
323/323 [==============================] - 33s 101ms/step - loss: 0.0110 - acc: 0.9968 - val_loss: 0.6360 - val_acc: 0.9217
Epoch 8/15
323/323 [==============================] - 3

In [10]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[7].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
#emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 186, 64)


In [0]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75271, 186, 64)
(75271, 8)
(16524, 186, 64)
(16524, 8)


In [0]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Embedding
from keras.layers import Conv1D, MaxPooling1D,GlobalAveragePooling1D

In [0]:
model = Sequential()                        
#model.add(Embedding(vocab_size, output_dim=emb_dim,input_length=max_length))
model.add(Conv1D(filters=256, kernel_size=7, input_shape=(X_train.shape[1],X_train.shape[2]),activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.4))
model.add(Conv1D(filters=128, kernel_size=6, activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [15]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 180, 256)          114944    
_________________________________________________________________
dropout_1 (Dropout)          (None, 180, 256)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 180, 256)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 180, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 175, 128)          196736    
_________________________________________________________________
dropout_3 (Dropout)          (None, 175, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 175, 128)         

In [16]:
history = model.fit(X_train, y_train, epochs=25,shuffle=False, verbose=1,validation_split=0.1)
#model.save_weights('./checkpoints/text_classifier_rnn_tfidf')
!mkdir -p saved_model
model.save('saved_model/text_classifier_rnn_tfidf')

Train on 67743 samples, validate on 7528 samples
Epoch 1/25
67743/67743 [==============================] - 18s 263us/step - loss: 0.1147 - acc: 0.9739 - val_loss: 0.0817 - val_acc: 0.9830
Epoch 2/25
67743/67743 [==============================] - 17s 256us/step - loss: 0.0827 - acc: 0.9811 - val_loss: 0.0743 - val_acc: 0.9831
Epoch 3/25
67743/67743 [==============================] - 17s 256us/step - loss: 0.0741 - acc: 0.9823 - val_loss: 0.0814 - val_acc: 0.9789
Epoch 4/25
67743/67743 [==============================] - 17s 256us/step - loss: 0.0670 - acc: 0.9839 - val_loss: 0.0944 - val_acc: 0.9772
Epoch 5/25
67743/67743 [==============================] - 17s 254us/step - loss: 0.0669 - acc: 0.9843 - val_loss: 0.0883 - val_acc: 0.9758
Epoch 6/25
67743/67743 [==============================] - 17s 256us/step - loss: 0.0634 - acc: 0.9847 - val_loss: 0.0872 - val_acc: 0.9823
Epoch 7/25
67743/67743 [==============================] - 17s 253us/step - loss: 0.0626 - acc: 0.9845 - val_loss: 0.1

In [17]:
results = model.evaluate(X_test, y_test)
print(results)

16524/16524 [==============================] - 2s 113us/step
[0.13115887461837514, 0.9842047691345215]


In [0]:
y_pred=model.predict(X_test)

In [19]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [20]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])
print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

98.43533249731475
98.52566559047088
98.47517049586634
